In [1]:
import codecs,math,re,glob

In [2]:
numLangue="01"
nbLots=3
nRepLot="/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Lots/"

In [27]:
def nomFichierLot(numLangue,numLot,charabia=True):
    if charabia:
        return "/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Lots/Extraits-%s-Lot%d-CH.text"%(numLangue,numLot)
    else:
        return "/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Lots/Extraits-%s-Lot%d-FR.text"%(numLangue,numLot)

def getConte(numLangue,lots,charabia=True):
    texte=[]
    filtrePonct=u":,;()\t- ’´'`…–‚«»"+'"'
    if not isinstance(lots, list):
        lots=[lots]
    for numLot in lots:
        with codecs.open(nomFichierLot(numLangue,numLot,charabia),"r",encoding="utf8") as input:
            texteChaine="\n".join([l.strip() for l in input.readlines() if l.strip()!=""])
        stops=re.escape(u".?!")+"\n"
        texte.extend([l.strip() for l in re.split(u"[%s]"%stops,texteChaine) if l!=""])
    if not charabia:
        tempTexte=[]
        for phrase in texte:
            tempPhrase=""
            for lettreBrute in phrase:
                if not lettreBrute in filtrePonct:
                    tempPhrase+=lettreBrute.lower()
                    if lettreBrute in "?!":
                        tempPhrase+=" "+lettreBrute
            tempTexte.append(tempPhrase)
        texte=tempTexte
    return texte

In [4]:
def getNgrams(texte,seuil=2):
    fNgrams={}
    tNgrams={}
    lNgram=1
    freqMax=10000
    epsilon=0.01
    while freqMax>seuil:
        tNgrams[lNgram]={}
        fNgrams[lNgram]={}
#        print lNgram
        for ligne in texte:
            for i in range(0,len(ligne)-lNgram+1,1):
                ngram=ligne[i:i+lNgram]
#                print ngram
                if not ngram in tNgrams[lNgram]:
                    tNgrams[lNgram][ngram]=0
                tNgrams[lNgram][ngram]+=1
        fNgrams[lNgram]={k: v for k, v in tNgrams[lNgram].iteritems() if v>seuil}
#        print lNgram,"=",len(fNgrams[lNgram])
    #    print ",".join([k for k in fNgrams[lNgram] if fNgrams[lNgram][k]>seuil])
        if lNgram>1:
            keys2=fNgrams[lNgram].keys()
            for ngram2 in keys2:
                keys1=fNgrams[lNgram-1].keys()
                for ngram1 in keys1:
    #                print ngram1,ngram2
                    if ngram1 in ngram2 and float(fNgrams[lNgram-1][ngram1]-fNgrams[lNgram][ngram2])/fNgrams[lNgram][ngram2]<epsilon:
                        del fNgrams[lNgram-1][ngram1]
#            print lNgram-1,"=>" ,len(fNgrams[lNgram-1])
        freqs=[fNgrams[lNgram][k] for k in fNgrams[lNgram]]
        if freqs:
            freqMax=max(freqs)
        else:
            freqMax=0
        lNgram+=1
#        print freqMax
    return fNgrams,tNgrams

In [5]:
def getHighNgrams(ngrams,longMin,freqMin):
    ngram2Freq={}
    freq2Ngram={}
    for l in ngrams:
        if l>=longMin:
#            print l
            for ngram in ngrams[l]:
                if ngrams[l][ngram]>=freqMin:
                    freq=ngrams[l][ngram]
#                    print ngram,freq
                    ngram2Freq[ngram]=freq
                    if not freq in freq2Ngram:
                        freq2Ngram[freq]=[]
                    freq2Ngram[freq].append(ngram)
    return ngram2Freq,freq2Ngram
    

In [6]:
def getData(numLangue,lots,longMin=3,freqMin=5):
    texte=getConte(numLangue,lots)
    fNgrams,tNgrams=getNgrams(texte,seuil=freqMin)
    return getHighNgrams(fNgrams,longMin,freqMin)

In [7]:
ngramFreq,freqNgram=getData("06",[1,2],4,3)

In [8]:
def sortie(text):
    print text
    outFile.write(unicode(text)+"\n")
    return

In [9]:
with codecs.open(nRepLot+"FreqNgrams.txt","w",encoding="utf8") as outFile:
    for l in ["%02d"%(n+1) for n in range(11)]:
        sortie("Charabia "+l)
        sortie("--------------")
        lots=[]
        for num in range(nbLots):
            numLot=num+1
            lots.append(numLot)
            print "Lot %d"%numLot
            sortie("<<<<<LOT%d>>>>>"%numLot)
            ngramFreq,freqNgram=getData(l,numLot,4,3)
            for f in sorted(freqNgram.keys(),reverse=True):
                sortie(f)
                for n in freqNgram[f]:
                    sortie(n)
                sortie("")
            sortie("")
        sortie("==========TOTAL=============") 
        sortie("")
        ngramFreq,freqNgram=getData(l,lots,4,3)
        for f in sorted(freqNgram.keys(),reverse=True):
            sortie(f)
            for n in freqNgram[f]:
                sortie(n)
            sortie("")
        sortie("")
        sortie("=================================") 
        sortie("")

Charabia 01
--------------
Lot 1
<<<<<LOT1>>>>>
16
qtir
uyrix

14
qtirix

12
ixed

11
izix
hizi

10
iqti
ixzi

9
ziqti
wadzi

8
rixw

7
iziq
tizi
iqxi
dwta
exzi
ñreq
izih
raqi
iqtir

6
xedi
ñixh
odix
ixxi
qtid
xziq
zizix
ziqtirix

5
zihw
azix
ossi
ytej
ziqñ
ñixhi
iziqti
xediha
wraqizi

4
ziqi
tazi
ddos
xzix
jadi
zixe
odxe
ihed
codx
iyte
rixr
dhoy
zixz
iqix
wsay
axzi
zied
dizi
whiz
ñreqp
hizix
hizih
royyi
ixraq
edwta
ziqxi
texzi
iphiq
iqñix
iuyrix
qtirixw
odixxic
xedihahtizi
wadziqtirix


Lot 2
<<<<<LOT2>>>>>
23
ixzi

21
qavuxgiy

16
ñixzi

15
ñisw

13
ixed
ñiswix

12
oyyi

11
ihed
adzi
ñarzi

10
ossi
iñux
ñreq
wadzi

9
iziz
oxdi
wtex
zizi
ecwh

8
caxz
taqx
htaq
axzi
iwtex
sexdi
xdiwt
roxzui
ijyecwhiq

7
ñtaq
xdiq
xdix
aqzi
xñar
sexi
teqh
payz
xzix
iqed
ñaxw
edwt
añax
izizi
aqxih
htaqx
sexdiwtex
syessijyecwhiq

6
ixwh
hoyñ
jyir
hhoy
ñteq
htix
xedy
xsex
hizi
royy
tixd
ajhi
taña
eqzi
iwta
tazi
izix
ixña
axdi
xraq
edzi
ozix
qziq
qizi
qzih
dzih
ñaxñ
iphiq
ziñux
teqhi
zityod
xñarzi
syahqi
ht

zxov
nqxe
aron
xovi
becu
vopa
itxo
kini
aqak
kaya
xoro
zkan
xogi
givin
tezni
xolxo
agnik
xekga
xizxe
izxoz
ivxoz
gotma
xekxe
kanozo
rxepxe

4
xojx
arih
mavo
ikxo
gini
upiz
anag
vulu
igni
anva
ikin
zxoi
goto
isxo
ztez
vogo
pxej
gmxo
bxog
volo
upuy
open
akoz
rono
ihxo
ozag
ezxe
zvig
xopi
vxiz
upen
onya
ozka
irit
ngaz
anvo
unvu
txoz
voka
upgu
okan
zagk
xewa
yuhmu
vxowa
zgnik
xokin
ezuzv
xoron
kizvx
imkin
igoim
jiyiz
omxey
nvigo
viyvi
kinig
ilxoz
inipi
pxokv
xotxo
penvu
lxozxo
aqakvo
givxoz
txipka
xokvxo
xohmxo
mxolxo
xozxoz
egivin
gafizam
nbeculu
tinipxoz
rihkivxo
agnikoba
mxeytinip


Lot 3
<<<<<LOT3>>>>>
39
ivxo

36
inxo

23
vxoz

22
xozx
ivxoz

21
nisuzjak

20
xepxe
givxoz

19
xozv
rxepxe

18
xozxo

17
xozvi
rxepxecxe

16
egiv

15
ilxo
qxin
kinxo
ginxo

14
inig
envu
txoz
ingxo

13
vopa
itxo
open

12
ozva
xegi
xotxo
penvu

11
xoti
xozi
rxepxecxegivxoz

10
izxo
xeja
xolxo

9
xipk
qxez
lozo
anoz
lxoz
volo
xoki
rong
izvx
snazt
kivxo
inxoz
xotxoz
tingxom

8
yapa
qazb
guka
kano
anvo
igmx
okin

123
wewe

112
pwer

50
wewer

47
epwe
lañu

43
epwer

42
weri
lañuz

41
urwe

40
rtwe
pweri

37
etne
ugen

36
zwewe

35
llwe

34
uzzu

33
anne

32
unti

30
zearu
tizwe

29
erte
ñuzz
erir

28
rnwe
nnwe
getn
wesu
lañuzzu

27
ugwe
werz
ñanne

26
tull
wern
unwe
zarur
nwewe
wewes
pwerir

25
rpwe
wert
tnwe
unnir
zwewes

24
urir
zwewesuwo

23
eolu
tunn
upur
iñwe
rpwer

22
wepwe

21
gwen
ñeer
wepwer
zarurve

20
rzea
iunt
ugwen
irtwe
upwer
getne
tunnir
zarurven

19
netu
nwer
utir
ttwe
wers
iunti
zarurvenex

18
nweñ
uzwe
eztu
urze
rzwe
tnet
pwern
untiñ

17
utne
ette
ñwer
werb
tero
irte
llour
tiñwe
tiuntiñ

16
wetu
neze
erza
ilwe
bnnw
tuge
wetn
tnetu
innwe
ullour
getnet
rzearu

15
nwep
elañ
urar
irze
rirz
ñuzu
erna
erwe
intw
suir
zaur
rnao
ñura
ñurwe
weñwe
wertwe
getnetu
tullour
zearupu
tiuntiñwe

14
nbnn
werr
erbn
tirt
etnu
npwe
uzur
ñette
ullwe
rlañu
lañuzu
zearuu
ñintwe
zearupur

13
tura
ulli
twet
oluz
zaze
tuur
rtun
outi
zeer
etur
etna
ñwewe
npwer
werbn
siulu
nbnnwo
wernwe

12
zbll
ñutn
tnau


erlañuzur
innwepwer
lañuzzuez
weunusaol
leennuell
ezzaretne
silwetnbu
zazeñurar
welañuzzu
tiuntiñwet
rtiuntiñwe
imirpolntu
pwertunnir
uzwewesuwou
rñbxzbllutir
tiuntiñwergao
ntiuntiñwerzwewesuwowe


==========TOTAL=============

243
wewe

198
pwer

104
wewer

87
epwe

80
ugen
epwer

79
rtwe
unwe

76
weri
anne

75
urwe

73
ugwe
wert

72
llwe

70
erte

68
etne

65
pweri

64
tnwe
ñanne

63
ñeer

55
erra
izwe
unni
zwewe

54
nnir
nnwe

52
tunn
tero
tizwe
unnir

50
gwen
urir
erir
wepwe

49
nwer

48
ttwe
lañu
ugwen

47
rnwe
uzzu
wepwer
tunnir

46
werz
wern
getn
nwewe

45
tull

44
rpwe
perr

43
zixn
unti
irte
werir

42
wetu
lañuz
perra

41
eolu
eper

40
utir
wertwe
pwerir

39
upur
upwer

38
rpwer

37
erou
lest

36
xnna
illi
getne
swewe

35
utne
eztu
nweñ
intw

34
nwep
tirt
ñintwe
zixnna

33
irtwe

32
wers
luge
wetn
uzwe
well
innwe

31
rzur
elli
urze
terou
zearu

30
netu
ertu
nwez
tnet
wesu
ilwe
eñee
ezze

29
utnw
ñuzz
erbn
zaur
llir
llour

28
ulli
rrwe
arur
werb
werñ
iñwe
ujwe
tuge
rnao
erza
eu

ruñi
orñb
irlu
iuni
iune
perñ
tiln
reol
noug
iluñ
irro
iññu
irñu
teña
ilun
lweu
lweg
lwei
luuz
nepi
utuz
tuñu
tuñi
utue
utug
upet
iwer
rpez
rtao
oxue
raar
otir
ituu
zzeg
exao
tega
toñe
rlup
wesb
eraa
erao
aann
azen
ñell
orpe
nesu
lute
letn
ezux
exul
reax
pnau
irpu
naoo
naos
ezaz
ezan
nnul
zeel
nnut
nnur
wefa
peaz
opwe
zatn
zaxu
zinn
rrou
tnae
utin
utii
uret
efot
efou
uñol
nene
utiñ
tiir
raza
tuwe
eefe
teeg
eutz
teez
taur
tauz
arzu
eetu
uzer
nzee
uune
lirn
luzo
ñerf
luze
eñeu
etau
rnop
zurl
nbwo
rgwe
zurr
toxñ
reut
ufeg
ziuñ
tnei
arsu
bzti
pnuu
zuge
urut
egiu
enñe
noñe
gell
inge
ritu
rriz
rrip
nirs
zeti
neto
nape
ññii
eurñ
nwot
euru
elen
eler
ñser
ulwe
eere
sepn
eeru
zofe
eann
epin
pela
ofir
ueze
rzal
eonn
rñez
perz
iuro
iuri
rnau
iroñ
pirv
nael
teze
irta
enpi
etue
eñul
lann
zien
unee
unen
izzu
rser
xzet
rswo
foll
rbnt
sexe
luir
nouñ
nexz
pwez
pwen
usut
nipj
elut
ireu
enep
jirt
ozel
ozeu
ofeñ
tuet
elge
ulzu
uzal
etil
exñu
ttur
ealo
utze
rfoa
brwe
lilu
uuxa
rfao
rzer
rzex
tirf
rgee
zaar


...

45
agag

41
tagc

28
agagc

22
mocec

20
llag
ucha

17
edal
agch
ulla

16
olla
chag

15
hlag
ssag
ecag
ahla
ecma

14
agcu
hhag

13
ehla
tagcu
mocecna

12
edag
agda
hess
agla
agcha

11
maac
agcl
elag
acha
lieh
ullag
etagc

10
ecca
lagm
amoc
clag
ehha
cmag
ussu
atagc
elluc
qolla

9
aghe
emoc
uhla
essa
qaac
emag
umag
ecuc
commo
vagag
lagag
secca
agchag
mocecmaac

8
etac
ahha
lama
lamo
seda
agcd
ahle
etec
haci
maga
gima
aise
homo
lagc
evag
humag
ctagc
aglag
tagcuc
agtagc
mocecnah

7
aghl
agga
ceec
hlaa
agmo
agco
hloe
momm
mimi
utuc
laac
elhu
eeca
aacl
mmea
ssuc
aacm
heec
acol
ussa
lagp
cmoc
ucma
essag
agcla
edagl
sedal
tagcl
uchag
liehla
poohomom

6
laco
aull
acie
aghu
hlam
agma
aclñ
isec
cagl
caga
lucm
hhac
agis
illa
daec
qagh
lull
doil
amah
aeci
sagi
upag
iecu
gagh
ucla
adoi
lada
emme
lose
lagq
lagl
lagd
oise
assu
aehl
acla
asla
adae
magag
qelag
viema
maemm
hlaac
vucha
heecag
helluc
commoe
cmocec
agclag
lagagc
mutpag
poohomomma

5
cuku
lñll
cahl
heda
hlac
mhuc
malh
agme
uqag
heca
agde
ag

qecoc
usmag
llagd
laguc
cmile
ahlie
ssagh
aggam
voopi
ahlev
lagla
qicla
lussu
laggi
amupl
ucvag
uccag
oedal
misqa
misqu
oseda
aacma
aglie
agoop
edale
aclie
helie
cheha
ledal
agñll
ehluc
eeche
lagmo
ceech
lluse
ucehe
uagag
lalñl
oossu
achal
achat
mupla
liehe
lagda
liehu
ssuva
dacta
cagel
eheec
llagm
iehha
agdap
laeda
hullu
ecage
ecaga
lleda
ipgac
lucha
chahu
eutoq
alose
elagu
vupag
aqama
agcis
ucmec
omaac
hugas
daglm
aslam
lameu
aacda
cuhla
edact
cveuc
cehag
agmos
agcee
mimit
mocca
giolo
cdasla
povuhe
ctudih
ghlagm
clatli
iehlam
dactuq
chhemo
aactag
cmaoce
eullag
aqelag
uhlaga
ulagag
vagagl
qagagc
agssag
duplag
mecise
cqolla
iehuch
luucha
adahla
ullaah
hecagl
lñligc
luehha
lhussu
ciemua
ectagc
lvagag
matagc
lagloi
llagag
agqagm
agehlo
humagm
maoceh
cmaemm
qamahe
miulla
cmolho
qisuqq
agmimi
ameuto
maehec
aehloe
mileal
amihma
tagche
ammahe
ladoil
ltagcu
ghumag
etñdag
ecossu
edalhe
maqape
qucelo
agagcm
lavagag
hellucl
cviemah
mosocol
ietagcu
gimaise
agchagg
agagcha
voisuss
emucnag
moslaeh


56
ayua

53
uasw

52
hayuasw

42
uhayuasw

29
polw
vovv

28
ilwi

27
suañ

25
fzua
uañw
bfuh

24
ovvu

23
uaju

22
viaf
suañw

20
svuh
cons

19
vuañ

18
mial
ivvi
onvv

17
milu
uass
vihu
osvu
iali

16
mañk
fzia
ffzo
hiaf

15
aswñ
añki
vuhu
vilw
wsua

14
zvua
svih
usua
qosv
ajuñ
vaziñ
yoazf
wsuañw
hayuaswñ

13
qial
ensv
ñiaz
ffzi
vuha
vvuh
polwu
uffzoñw
hayuaswsuañw

12
zual
afmi
affz
iñki
muay
iafz
siar
fziy
uzvu
silw
isow
ivvu
joas
yfzua
vovvu
yoazfu
bfarhi
fuffzoñw
hayuaswñia

11
ajiñ
unvo
usum
qusu
fial
iafm
kvuh
uañu
iasw
vuah
lwiv
onyil
ekquy
uzoyf
mañki
uhayuaswñ
uhayuaswsuañw

10
uvvu
vvia
iaji
zivv
ufzu
juñu
iñha
uayu
wfzi
yens
kivv
iñvu
uñim
vmil
ekvu
asvu
miñv
iayv
ikiv
fmil
nsvi
ilwiv
uajuñ
oswuñ
uzvuañ
hayuaswca
bfarhiyoazf

9
isvi
suav
zony
liaf
uajj
lwfz
moñk
fzoa
vmañ
ivia
kvih
iyih
zviñ
iafu
oqua
nuay
ifzv
mañv
onye
ovvi
honz
vvuu
lwuñ
ilwf
uavo
azvi
usuañ
iajiñ
silwi
afmil
ikivv
rikal
vvilw
wfziy
upolw
zekquy
oyfzual
uhayuaswñia

8
vusu
esvu
uaso
ensi
hens
hmil
sivv
af

uhis
yimi
uzus
oaho
olvu
imyi
fuñi
knia
juñp
juñf
hñuh
iañv
iañj
uafu
ñbfi
uaff
yvih
evuv
iuzi
ñuzu
afho
ñhaf
yfua
iuhu
suaj
abfo
ovvc
yfuñ
ujon
ayow
bfon
ruañ
yikv
bfah
afyu
posv
iñen
ihau
viha
faho
isiu
puay
iahj
suñu
nvon
aubf
mily
lwbf
milo
ivif
ñviv
vfia
anvf
iñbf
svim
vñia
sviu
uziy
zviv
ñens
muav
wuñm
wuñh
konl
colu
uhuv
kanv
konñ
alwi
kuay
lwyi
uñuv
imis
nshi
fopu
wuas
usov
vpoz
ihuh
vvon
ihuz
vuci
omoy
imuz
yumi
ifzo
vhon
ialñ
lwvi
uahj
uahi
uaho
nvuv
hvua
ahju
ialp
hfzu
kilw
hoym
aiaf
uvñu
ivvo
ivvm
ñmia
mjuñ
lfzi
habf
roah
zoav
ilwq
uhañ
ilwm
huju
avuz
avuh
knum
knuv
ilwñ
iiaf
peaz
lwia
milv
mils
vviñ
iñon
iañm
nenh
mule
ehoñ
vuma
añvo
jivi
avfi
ponz
uane
uzen
uzea
ñwuv
uahñ
muma
añku
ñuro
iñis
fiaf
viaq
sfzo
fzif
ulwv
huha
uzvf
ukos
ymuk
vuru
viaj
ahos
ikah
svuñ
ñvuf
ucis
añhu
jiñu
jiñw
jiñk
iafy
jiñf
hiañ
yuau
uvua
yual
ñbfu
huyi
iffi
simi
ajju
lwji
mavu
rufz
añih
añim
owaf
vvvo
hevu
jjiñ
onñv
afqi
viñi
svfi
vmua
alia
kial
uven
vven
aliñ
hmiy
avpo
lukv
iyik
iyiñ
suhu
iafv


etyue
wdmer
ñuvet
eubbe
ebuek
birfe
nratb
mueket
etqxim
xakbet
jikeyn
ejikey
gatbet
datutl
ñkaekn
ermeek
eñkaek
eqatutl
bejkutn
beaobnre
eutbeyuvq


Lot 3
<<<<<LOT3>>>>>
59
diik

45
etbe

39
etet

38
tiik

37
etfe

34
nrek

33
aenr

32
raet

30
nwda

28
enwd

27
aenre

26
wdre
utbe

25
iwdr
inne
itbe
iikb

23
etbi

22
innejaenrek

21
aebe
tuer
lutl
enwda

20
qxim
eket
nwdaet

19
miik
tbek
iikd
iikbe
ejkutn

18
iyye
bejkutn

17
biik
ebek
fuvt
ebou

15
eube
rxee
bitn
iike
imet
xeek

14
nrue
gaak
aley
tbei
iket
maau
beke
tetb
etfeu
tdiik
qxime
maebek

13
iikñ
awdr
ñkou
iikr
qjuq
itbek
galey
nruez
rdiik
benwda
qximet
enwdaet

12
beqa
aobe
rekn
tlut
yuvq
etiy
etra
etme
erme
etdi
traet
iikdi
maaue
etetb
wdrek
juqjuq
meunve

11
kuej
ekbe
ebet
etxe
doun
enru
retb
bein
merd
bede
bedi
xuybe
maenr
tiikñ
etraet
bouzven

10
aetf
kbet
eetn
yiik
ewdr
urre
iikf
ñaam
emit
tbit
kdiik
eñkou
ñkoub
eyrve
tiikd
fonrek
enruez
itbeke

9
uemi
aetb
ouge
etku
bega
eqye
nete
tbeb
etfu
feyi
keet
betb
betf
eyii
lao

In [10]:
with codecs.open(nRepLot+"LongNgrams.txt","w",encoding="utf8") as outFile:
    for l in ["%02d"%(n+1) for n in range(11)]:
        sortie("Charabia "+l)
        sortie("--------------")
        for num in range(nbLots):
            numLot=num+1
            print "Lot %d"%numLot
            sortie("<<<<<LOT%d>>>>>"%numLot)
            ngramFreq,freqNgram=getData(l,numLot,4,3)
            for n in sorted(ngramFreq.keys(),key=len,reverse=True):
                if len(n)>5:
                    sortie(u"%s %d"%(n,ngramFreq[n]))
            sortie("")
        sortie("==========TOTAL=============") 
        sortie("")
        ngramFreq,freqNgram=getData(l,lots,4,3)
        for n in sorted(ngramFreq.keys(),key=len,reverse=True):
            if len(n)>5:
                sortie(u"%s %d"%(n,ngramFreq[n]))
        sortie("")
        sortie("=================================") 
        sortie("")

Charabia 01
--------------
Lot 1
<<<<<LOT1>>>>>
xedihahtizi 4
wadziqtirix 4
ziqtirix 6
qtirixw 4
odixxic 4
wraqizi 5
iuyrix 4
qtirix 14
iziqti 5
xediha 5

Lot 2
<<<<<LOT2>>>>>
yazzohñtaqpayzixiz 4
qavuxgiyqavuxgiy 4
isyessijyecwhiq 5
syessijyecwhiq 7
sexdiwtexxix 6
ixsyahqizi 5
htaqxihed 4
pyihxoxdi 4
sexdiwtex 7
ijyecwhiq 8
hoyñixzi 4
wadziñux 5
qavuxgiy 21
ixñarzi 4
ñiswixs 4
roxzuih 5
htaqxih 6
yirizi 4
ñixziw 5
xñarzi 6
syahqi 6
wserix 4
ñiswix 13
zityod 6
caxzix 4
oxsexi 4
hñixzi 5
erixed 4
ñteqhi 4
dzañax 4
dwadzi 5
roxzui 8
qtajhi 4

Lot 3
<<<<<LOT3>>>>>
sexdixedzqexxoxdix 4
sexdiwtexxi 4
ñiyiñeppih 4
edzqexxoxd 5
zqexxoxdix 6
heqxiqewi 4
sexdiwtex 5
zqexxoxd 7
ñtissix 4
wyehhih 4
sexdix 6
zahhiq 4
hoywoz 4
ziqraq 4
ñarziw 4
xdixed 5

==========TOTAL=============

yazzohñtaqpayzixiz 4
sexdixedzqexxoxdix 4
qavuxgiyqavuxgiy 4
isyessijyecwhiq 5
syessijyecwhiq 7
sexdiwtexxix 9
wadziqtirix 4
sexdiwtexxi 10
xedihahtizi 4
ocozyiqhoz 4
ñiyiñeppih 4
ixsyahqizi 5
edzqexxoxd 5
zqexxoxdix 6

==========TOTAL=============

nizarurvenexoonnizearupurnwewellwe 5
zirñeeluñetteorzearupur 7
nopnoxuzeuzzunuinwegwe 4
ntiuntiñwerzwewesuwowe 4
nizarurvenexoonnize 7
tiuntiñwerzwewesuwo 5
terosuiruzagetnur 7
zearuuñsuguilwe 5
perrafezixnna 6
ñintwetnwefwe 6
tiuntiñwergao 4
liusweseletir 4
irtuññweurir 4
rñbxzbllutir 4
werzwewesuwo 6
uzwewerzaur 5
rzarurvenex 6
uzwewesuwou 4
iñeerxasitu 4
epwertunnir 4
tullourpwer 6
untiztirtwe 4
elweztugwen 4
erzwewesuwo 7
weroaneñeer 4
ñbxzbllutir 5
izarurvenex 8
zarurvenexo 8
ñintwetnwe 7
ñanneiuswe 4
paoripitir 7
tiuntiñwet 4
ñintwepwer 6
liugwerñax 4
tiuntiñwer 11
uzwewesuwo 5
eroaneñeer 6
rtiuntiñwe 4
piunnwerbn 4
nnweñwewer 4
nusilwetnb 4
imirpolntu 4
ñeerxasitu 5
rzwewesuwo 10
ñintwettwe 5
einnwepwer 4
zarurvenex 19
nelorsouze 8
ugelzourir 4
ñanneperra 4
etiuntiñwe 6
ezearuuñsu 5
liugwetnwe 4
tunnirpwer 6
lweztugwen 5
zwewesuwon 9
pwertunnir 7
irwewertee 4
zartunnir 6
irtuññweu 7
ñintwewer 9
getnetuge 4
weunusaol 5
nnbwertwe 4
silwetnbu 5
zwewesu

gadbobbazxñiaggañ 6
zisgazxñiaggañad 9
wobzxñiaggañad 5
gazxñiaggañad 11
eszxñiaggañad 7
zxñiaggañade 4
zxñiaggañadz 9
azxñiaggañad 13
zxñiaggañadv 6
zhylagawzbis 8
zxñiaggañad 43
adfmauñdods 4
ologbañwog 4
azxñiaggañ 19
zxñiaggañ 50
fidzxubba 4
xialqañda 5
wobxedsad 5
veñbidswa 4
adbobbaz 8
ziegidad 4
exialqañ 5
gozxeyad 4
fidfiyga 4
axialqad 5
gadidgad 5
fidxudda 7
gadbobba 9
afeddods 4
kidsaveñ 4
miasañda 4
zisgafid 8
zxeyades 4
esgifid 4
ugadveñ 5
azillad 4
ozxeyad 6
agadzie 4
dzxubba 5
isgafid 9
ziegida 5
feddods 5
soyafil 4
awavwañ 4
axialqa 7
wiadxwa 6
xialqad 15
gañoxxa 4
xialqañ 9
xizwaga 4
gawyiñg 4
adbobba 11
gawzxib 4
zyiñaga 5
adfiyga 4
añzisga 4
eszisga 8
oadviñw 4
sñadada 6
añhwwañ 4
adoxxa 4
xedsad 14
awugiv 4
wñiaaw 8
gañadf 4
sxizwa 4
vauñzw 4
qieads 4
didgad 6
gadveñ 6
asañda 5
yogaña 4
xuddaz 4
zqñids 4
aveñad 4
liewwa 4
gauñad 4
xialqa 26
bellad 4
añades 4
eddods 7
gañveñ 5
wobiwz 4
adzief 5
agagad 6
adsids 4
qiegad 7
ziebag 4
gawyiñ 7
ziegid 6
gañqie 6
dodsad 4
sz

jwiopi 4

==========TOTAL=============

foifziopahfimimyuuzi 4
quyzimumwlazwiwiiz 4
zucemkihzucemkih 4
fiomyuuzyizemwiz 4
pahfimiuñcqihwz 4
fujpheñbgjgomf 6
fizgañmopjjaym 6
foiihhizvewwiz 5
pahfimimyuuzi 5
pahfimiuñcqih 5
mobywhumpija 4
imumwlazwiwi 5
umwlazwiwiiz 13
ihhizvewwizj 4
ifziopahfimi 6
viomiwabywiz 4
iihhizvewwiz 6
fizluñbywiz 5
pheñbgjgomf 9
jumwlazwiwi 4
ihhizvewwiz 7
fimxzoiqemf 5
foikuexizom 11
mumwlazwiwi 6
gañmopjjaym 7
eñxizquyzim 4
imzucemkihm 4
wifizwieqih 4
mobywiomvuh 6
umwlazwiwif 6
fizwieqihl 4
fijwieqihj 4
fizgañmopj 8
fijgañmopj 4
iomimxzoiq 4
imzucemkih 5
ifizwieqih 7
jfizgañmop 4
umwlazwiwi 25
vubywijoby 5
umwlazwiw 26
foigañmop 4
jczubyjoi 5
jbyubywih 4
fizwieqih 14
emfjczuby 7
quñyzvumm 8
izucemkih 6
hzucemkih 6
fizgañmop 20
mzucemkih 6
zucemkihm 8
rizpmeñpw 4
zluñbywiz 6
ikuexizom 12
piwzuñevw 6
luñbywiz 7
izjbyzug 4
mobywiom 7
obyliojj 4
fizgmuxi 5
xiyuhwim 4
wkwmobyw 4
qhibywim 4
gañmopom 6
izvewwiz 8
emfuhjiz 5
japhioby 4
emfjupwi 5
izoyvfoi 4
ijojwio

eyekeyajkzucpuppucbucricjeñbewefotmuq 4
xezyitraotqjuburucbucbewefotmuqi 6
aotqjuburucbucbewefotmuqi 7
jzeqiduyucfotmuqipucpic 5
pucbucricjeñbewefotmuq 5
ehceehcepothqurece 7
jecfirijitzyitmit 4
hubigotgucicuxicu 4
ucfotmuqipucpic 7
keyajkzucpupih 4
keyajkzucpupic 5
ukeyajkzucpupi 5
icgipijiyruciy 4
eckeyajkzucpup 4
cbucbewefotmuq 9
keyajkzucpupr 4
keyajkzucpupi 14
ukeyajkzucpup 6
ekeyajkzucpup 12
bihupupqucbi 4
jezfiriruhem 4
hubigotgucic 7
keyajkzucpup 38
bihupothquc 4
jebezfotmuq 4
jotdotjjitm 7
jitzyitmit 9
bewefotmuq 14
ichecitcbe 4
guxekxuzbi 4
jebitcitc 4
bihupozba 5
fotzotcgu 6
itcjeznit 4
otxuheccu 4
hubigotgu 11
idicbujeq 4
fotmuqriz 4
ucfotmuqi 8
ujekithit 4
otzotcbot 4
itqitsitc 4
uguxekxuz 4
oqbasaca 4
jekithit 7
jzeqiduh 5
jiyruciy 8
itsitcbe 5
ucfotmuq 9
gisuzici 4
jitpjecd 4
edqezitc 4
itjebitc 5
jzeqiduy 7
pedqezit 5
yujipñuy 4
qezitcit 6
ehcecitc 6
jucbipic 4
guxekxuz 5
tjebitcj 4
izeweñuñ 4
gotguquz 12
otcothoz 4
heñesitc 4
qithozba 4
heñesit 6
upothqu 5
guxerit 4
es

neañya 4
kojaaw 4
nayaaw 4
beabaw 5
wesaaw 4
masiwñ 4
aawkaa 4

Lot 3
<<<<<LOT3>>>>>
mabewowsawmabewows 4
awmabewowsow 5
sawmabewows 5
meetwteeñja 7
awmabewows 10
mabewowsow 6
makagakex 4
mabewows 15
bewowsaw 5
awmakaga 4
ahtowñañ 4
bewowsow 7
ahtowña 5
majewsa 4
ñgoahaw 4
awñgoah 4
mahtowñ 5
ratmat 5
higaoñ 7
ñgihaw 4
ahtowñ 8
letmaw 4
bewows 17
awigga 4
awawma 7
ameetw 5
awmiit 4
awmabe 12
xaalaw 4
iitaaw 4
htowña 7
nayaaw 4
ewmatm 5
makaga 5

==========TOTAL=============

mabewowsawmabewows 4
stojdaniwwayja 5
mabewowsawma 5
awmabewowsow 5
meetwteeñja 7
kayniwwayja 4
sawmabewows 5
mabewowsow 6
sowsratmat 4
iiwsatiiby 5
wiulagojbñ 4
awmabewows 11
wiitkayzeñ 5
iwyleetmma 4
wmabewowsa 4
bewowsawm 6
wmabewows 14
mahtowñañ 5
amameneet 4
mabewowsa 8
yawñgeyya 4
makagakex 4
mabewows 22
niwwayja 10
wmevkyat 5
ahiñyeet 4
doyyawaw 4
bewowsow 7
hawawkoj 4
bewowsaw 7
awkojgoa 5
maylaama 4
awmakaga 5
mameneet 17
ahtowñañ 6
woaygiws 4
dojwetaw 4
awawyeaw 4
maneamat 6
woaynaat 5
kaybowm 6
aawñaaw 4

ñiuknuvohvonjjfziañuav 4
penqiswhmuvwa 12
ufzosluasoa 4
usoacensiaj 5
hauasvovvu 5
iakifuruam 6
iakifurnu 4
jfziañuav 5
fzusukvuh 4
hauasvovv 6
uzuffzoñw 6
uhumyuaju 4
upenqiswh 4
penqiswh 14
humyuaju 5
uyujsoah 4
uzimjiyu 4
vuzvmoñk 4
pussovvu 4
haujuasv 4
polwbfuh 4
uañvuaf 4
ucuyhuf 5
uhumyua 5
ilwfziy 7
uzvuañq 4
uvipiañ 4
ufzensi 4
moñkvuu 4
vuhumyu 4
censiaj 6
lwoyfuñ 4
uzilwah 5
vuzvuañ 4
hizkivv 4
haujuas 6
vimuvwa 4
fziañua 7
iakifur 10
munkiañ 4
suaymua 4
vumulua 5
uzimjiy 10
uzvmoñk 5
cisisvi 5
anvovvu 4
uzunvoa 4
cuyhuf 10
hiyens 4
iakifu 11
humyua 7
imjiyu 5
jjfzia 5
moñkvu 11
luajuñ 4
hmuvwa 13
qialyi 4
ñulwov 4
uzvuañ 7
sivivv 4
fziañu 8
uhusur 4
iajows 6
qasual 6
iarriñ 4
vipiañ 5
upuazu 4
fzensi 5
polwmi 4
jiñkow 4
ovvilw 4
nvovvu 6
loñkos 4
yujsoa 6
uhumyu 10
jazoñw 6
ilwfzi 9
uhobfa 5
isisvi 6
ussovv 5
anvovv 6
ucinia 4
iliañw 4
issivv 5
vovvuh 4
vvilwi 4
lwfziy 10
muafzo 4
hauasv 7
muvwam 5
suaymu 5

Lot 3
<<<<<LOT3>>>>>
uhayuaswmiluhayuaswñia 4
uhayuaswhafuffzoñw 5

juqjuqjuqjuqet 4
enruezfonrekn 4
rakreybouzven 4
etinnejaenrek 4
laobetfuygek 4
tiikñuttetet 4
tinnejaenrek 6
linnejaenrek 4
innejaenrekl 4
innejaenrekf 4
maaueqyeket 5
bebougetruy 4
nruezmaebek 6
innejaenrek 22
tiikñetebet 4
bedifeyiik 7
ewdreñkoub 4
etñelattet 4
etbeitbeke 4
utbenwdaet 4
qjuqjuqjuq 5
diikmaebek 4
kaeqebeqae 6
iikbejkutn 5
vedeñrleet 4
rgaleyrve 5
befonrekn 5
tbebouget 5
tbenwdaet 6
tbeitbeke 5
nwdaetrve 5
xikkeybet 4
iyyegaley 4
dermeunve 7
juqjuqjuq 8
enwdareyn 4
bouzvenet 4
egaleyrve 4
tiikrñiy 5
etraetbe 4
beqatutl 5
beitbeke 7
rawdtuer 4
yutfetut 4
benwdaet 9
rtuemitb 4
etutbein 4
difeyiik 8
ebouzven 7
enwdarey 6
maebekfe 4
etfuygek 5
lutltiik 6
tiikdiik 7
biwdrbir 4
bejkutnf 4
bebouget 7
bemaebek 4
tuermeek 4
galeyrve 9
etmaaue 4
eyutfet 4
etfeube 4
ketkuej 5
bejkutn 18
merdiik 7
leetqye 4
tuemitb 6
qetiyye 4
etbebou 4
ñketlet 4
eqatutl 7
ñaamjve 4
berxeen 4
diikeet 4
kqximet 5
efonrek 6
nwdarey 7
ebouget 8
bedifey 8
maenret 4
fonrekn 9
belaebe 4
tuermee 5
qjuqj

In [11]:
def getDistRL(ngrams):
    distLeft={}
    distRight={}
    for ngram in ngrams:
        bgramRight=ngram[:-1]
        lRight=ngram[-1]
        if not bgramRight in distRight:
            distRight[bgramRight]={}
        distRight[bgramRight][lRight]=ngrams[ngram]
        bgramLeft=ngram[1:]
        lLeft=ngram[0]
        if not bgramLeft in distLeft:
            distLeft[bgramLeft]={}
        distLeft[bgramLeft][lLeft]=ngrams[ngram]
    return distRight,distLeft

In [12]:
def calculEntropie(dist):
    total=sum(dist)
    result=0
    for element in dist:
        p=float(element)/total
        result-=p*math.log(p,2)
    return result

In [13]:
def getEntropy(dist):
    entropie={}
    for ngram in dist:
        entropie[ngram]=calculEntropie([dist[ngram][k] for k in dist[ngram]])
    return entropie

In [14]:
def getEntropieNgrams(numLangue,lots,charabia=True,freqMin=2):
    texte=getConte(numLangue,lots,charabia)

    fNgrams,tNgrams=getNgrams(texte,freqMin)

    entropyRight={}
    entropyLeft={}
#    lNgrams=getNgramsByLength(tNgrams)
    lNgrams=tNgrams
    for l in lNgrams:
        if l>1:
            distRight,distLeft=getDistRL(lNgrams[l])
            entropyRight[l-1]=getEntropy(distRight)
            entropyLeft[l-1]=getEntropy(distLeft)
    return entropyLeft,entropyRight

In [16]:
numLangue="01"
lots=[1,2,3]
strLots="-".join([str(l) for l in lots])

In [17]:
entropyLeft,entropyRight=getEntropieNgrams(numLangue,lots,freqMin=0)

In [18]:
seuilEntropie=2

In [19]:
with codecs.open(nRepLot+"Extraits-%s-Lots-%s-EntropyGauche.txt"%(numLangue,strLots),"w",encoding="utf8") as outFile:
    sortie("Charabia %s, Lots %s"%(numLangue,strLots))
    for k in sorted(entropyLeft,reverse=True):
        if k>1 and [n for n in entropyLeft[k] if entropyLeft[k][n]>seuilEntropie]:
            sortie("%d"%k)
            entropyK=entropyLeft[k]
            for e in sorted(entropyK,key=entropyK.get,reverse=True):
                if entropyK[e]>seuilEntropie:
                    sortie("_%s %f"%(e,entropyK[e]))
            sortie("")

Charabia 01, Lots 1-2-3
15
_isyessijyecwhiq 2.321928

14
_isyessijyecwhi 2.321928

13
_isyessijyecwh 2.321928

12
_isyessijyecw 2.321928

11
_isyessijyec 2.321928
_sexdiwtexxi 2.251629

10
_isyessijye 2.321928
_sexdiwtexx 2.251629

9
_isyessijy 2.321928
_sexdiwtex 2.250000

8
_qavuxgiy 2.355389
_isyessij 2.321928
_sexdiwte 2.250000

7
_ixñarzi 2.405639
_qavuxgi 2.355389
_isyessi 2.321928
_sexdiwt 2.250000

6
_ñiswix 2.732159
_whqasw 2.584963
_ixjyir 2.521641
_ixñarz 2.405639
_qavuxg 2.355389
_isyess 2.321928
_izizix 2.251629
_whoyyi 2.251629
_zityod 2.250000
_sexdiw 2.250000

5
_ñixzi 2.880838
_izizi 2.815072
_edwta 2.807355
_ñiswi 2.732159
_sexdi 2.655639
_iphiq 2.646439
_ixziq 2.641604
_whqas 2.584963
_wadzi 2.556263
_peqah 2.521641
_ixjyi 2.521641
_ixroy 2.500000
_hizix 2.500000
_uyrix 2.411602
_ixñar 2.405639
_qavux 2.355389
_diqxi 2.321928
_isyes 2.321928
_iodix 2.321928
_ihedw 2.321928
_hizih 2.251629
_hiñux 2.251629
_whoyy 2.251629
_wtazi 2.251629
_ñtaxz 2.251629
_ixciz 2.251629

In [20]:
with codecs.open(nRepLot+"Extraits-%s-Lots-%s-EntropyDroite.txt"%(numLangue,strLots),"w",encoding="utf8") as outFile:
    sortie("Charabia %s, Lots %s"%(numLangue,strLots))
    for k in sorted(entropyRight,reverse=True):
        if k>1 and [n for n in entropyRight[k] if entropyRight[k][n]>seuilEntropie]:
            sortie("%d"%k)
#            print k
            entropyK=entropyRight[k]
            for e in sorted(entropyK,key=entropyK.get,reverse=True):
                if entropyK[e]>seuilEntropie:
#                    print e+"_",entropyK[e]
                    sortie("%s_ %f"%(e,entropyK[e]))
            sortie("")

Charabia 01, Lots 1-2-3
14
syessijyecwhiq_ 2.235926

13
yessijyecwhiq_ 2.235926

12
sexdiwtexxix_ 2.419382
essijyecwhiq_ 2.235926

11
exdiwtexxix_ 2.419382
ssijyecwhiq_ 2.235926

10
zqexxoxdix_ 2.584963
xdiwtexxix_ 2.419382
sijyecwhiq_ 2.235926

9
qexxoxdix_ 2.584963
diwtexxix_ 2.419382
ijyecwhiq_ 2.113283

8
qavuxgiy_ 3.263933
exxoxdix_ 2.584963
iwtexxix_ 2.419382
jyecwhiq_ 2.113283

7
avuxgiy_ 3.263933
xxoxdix_ 2.584963
wtexxix_ 2.419382
ixñarzi_ 2.250000
yecwhiq_ 2.113283

6
vuxgiy_ 3.263933
xoxdix_ 2.807355
dwadzi_ 2.725481
ñiswix_ 2.565448
qtirix_ 2.521928
texxix_ 2.419382
ixzied_ 2.419382
xñarzi_ 2.413088
xwadzi_ 2.321928
tidroy_ 2.321928
erixed_ 2.321928
iñarzi_ 2.321928
waccix_ 2.251629
izizix_ 2.251629
whqasw_ 2.251629
zityod_ 2.250000
ecwhiq_ 2.113283

5
uxgiy_ 3.263933
iphiq_ 3.121928
wadzi_ 3.098460
ñarzi_ 2.968152
ixziq_ 2.947703
oxdix_ 2.845351
hizix_ 2.750000
tixdi_ 2.750000
ssizi_ 2.750000
ixhoy_ 2.725481
axzix_ 2.725481
secci_ 2.721928
zizix_ 2.641604
royyi_ 2.625815
u

In [21]:
seuil=2.5
lots=[1,2,3]
for numLangue in ["%02d"%(n+1) for n in range(11)]:
    print "Charabia",numLangue
    entropyLeft,entropyRight=getEntropieNgrams(numLangue,lots,charabia=False)
    for longueur in sorted(entropyLeft,reverse=True):
        if longueur>2:
            for ngram in entropyLeft[longueur]:
                if ngram in entropyRight[longueur]: 
                    if entropyLeft[longueur][ngram]>seuil and entropyRight[longueur][ngram]>seuil:
                        print ngram
    print

Charabia 01
dansla
quelle
avait
comme
était
èrent
aitde
aient
dela
pour
desa
rait
tant
elle
mais
tour
tout
donc
eset
equi
era
end
ene
ent
isa
ire
une
che
ait
ais
ela
ele
eet
ère
son
ons
ant
ain
par
ces
ale
ala
ten
ter
der
des
den
ite
tre
ier
set
ses
ser
sen
rie
een
per
sde
ler
les
cha
use
eux
our
res
que
qui
ure
ede
lui
eta
ils
est
ese
esa
eso
eau
ile

Charabia 02
dansla
epour
avait
était
èrent
aient
ante
rait
leur
quet
quil
elle
plus
eset
edes
equi
esau
unep
ait
que
ers
ere
era
ise
ras
ter
ire
ile
ils
ela
ele
eet
ven
ère
son
ant
par
ces
ala
ten
der
des
pro
ite
ien
sen
cha
ide
sde
tra
tre
len
les
let
ret
qui
ser
ure
ede
lui
eta
une
est
ese
esa
eau

Charabia 03
lesnains
laporte
margot
epour
avait
èrent
comme
erons
snous
faire
eelle
était
aient
eavec
telle
dela
avec
eles
ilse
sent
pour
onsa
lese
desa
sles
eret
esde
leur
ecou
elle
itet
beau
lait
plus
eset
isde
aut
ait
ner
nes
nep
mes
ere
era
ena
end
ent
lad
lac
lat
éla
isa
rai
ras
ter
ira
ire
cou
una
ils
and
ela
che
ele
eet
son
sou
ven
ae

In [78]:
numLangue

'11'

In [22]:
def getPhraseScores(phrase,lNgram,entropyLeft,entropyRight,somme=False,debug=False):
    entropieVect={}
    longueur=len(phrase)
    for pos,l in enumerate(phrase):
        if pos>0:
            if lNgram-1 < pos < longueur-lNgram:
                if somme:
                    if debug: print phrase[pos-1],entropyRight[lNgram][phrase[pos-lNgram:pos]]+entropyLeft[lNgram][phrase[pos:pos+lNgram]],
                else:
                    if debug: print phrase[pos-1],entropyRight[lNgram][phrase[pos-lNgram:pos]],entropyLeft[lNgram][phrase[pos:pos+lNgram]],
                
                entropieVect[pos]=(entropyRight[lNgram][phrase[pos-lNgram:pos]],entropyLeft[lNgram][phrase[pos:pos+lNgram]])
            elif pos <= lNgram-1:
                if somme:
                    if debug: print phrase[pos-1],"???",entropyLeft[lNgram][phrase[pos:pos+lNgram]],
                else:
                    if debug: print phrase[pos-1],"???",entropyLeft[lNgram][phrase[pos:pos+lNgram]],
                entropieVect[pos]=(0,entropyLeft[lNgram][phrase[pos:pos+lNgram]])
            elif pos>=longueur-lNgram:
                if somme:
                    if debug: print phrase[pos-1],entropyRight[lNgram][phrase[pos-lNgram:pos]],"???",
                else:
                    if debug: print phrase[pos-1],entropyRight[lNgram][phrase[pos-lNgram:pos]],"???",
                entropieVect[pos]=(entropyRight[lNgram][phrase[pos-lNgram:pos]],0)
    if debug: print phrase[-1]
    return entropieVect

In [24]:
import numpy as np
from scipy.signal import argrelextrema

numLangue="01"
lots=[1,2,3]

entropyLeft,entropyRight=getEntropieNgrams(numLangue,lots,freqMin=0)

def getFrontieres(texte,lNgram,side="both"):
    entropieVect=getPhraseScores(texte,lNgram,entropyLeft,entropyRight,somme=True)
    if side=="right":
        [frontieres]=[i for i in argrelextrema(np.array([entropieVect[e][0] for e in entropieVect]), np.greater)]
    elif side=="left":
        [frontieres]=[i for i in argrelextrema(np.array([entropieVect[e][1] for e in entropieVect]), np.greater)]
    elif side=="both":
        [frontieres]=[i for i in argrelextrema(np.array([entropieVect[e][0]+entropieVect[e][1] for e in entropieVect]), np.greater)]
    return frontieres

In [35]:
def printTexte(texte,frontieres):
    result=""
    for pos,l in enumerate(texte):
        result+=l
        if pos in frontieres:
            result+=" "
    return result

In [32]:
numLangue="01"

In [ ]:
lNgram=3
lots=[1,2,3]
for numLangue in ["%02d"%(n+1) for n in range(5,11)]:
    print "Charabia",numLangue
    with codecs.open(nRepLot+"Extraits-%s-Lots-%s-DecoupesEntro.txt"%(numLangue,strLots),"w",encoding="utf8") as outFile:
        sortie("Charabia %s, Lots %s"%(numLangue,strLots))
        sortie("")
        entropyLeft,entropyRight=getEntropieNgrams(numLangue,lots,freqMin=0)
        print "Entropy calculated"
        for texte in getConte(numLangue,lots):
            texte=texte.strip()
            if len(texte)>lNgram+1 :
                print texte
                entropieVect=getPhraseScores(texte,lNgram,entropyLeft,entropyRight,somme=False)
                frontieres=getFrontieres(texte,lNgram,side="both")
                sortie(printTexte(texte,frontieres))
            elif texte!="":
                sortie(texte)            
            sortie("")


Charabia 06
Charabia 06, Lots 1-2-3

Entropy calculated
yimpuwbhfunntuwuhntuwuruwbñhixtemepihsuwuxath
yimp uwb hfun ntuw uh ntuw ur uwb ñhi xt em epi hsuwu xath

hfunpulimbirwippeyomtewirtuhjorxowuwbuhuwbuxawmttenp
hfunpu limb ir wipp eyo mte wirtuh jorx ow uwb uh uwb ux awm ttenp

simunnuwirximpsimjuwenneyewbezumsitqtteptuqteprtufpueftuwuryewberechaweneqpterñtepezimbtipawqnturhihujarepqifippjorxowyimpmteproyemtite
sim un nu wirx imp sim juw enne ye wb ezum sitq ttep tuq tep rtuf pu ef tuw ur ye wb erec haw eneq pterñ tep ezimb tip awq ntu rhi hu ja rep qif ipp jorx ow yimp mte pro yem ti te

yterxinmtehehinteruwbhfunuwbnurwupuwrewbunxitluptuyuxp
yter xin mteh ehin ter uwb hfun uwb nurwu puw re wb unxit lu ptuyu xp

qopaxtejotipelimbirwippjtenjewterbiherezteqqiwfp
qop ax tejo tip el imb ir wipp jten jew te rb ih er ezteq qiwfp

emuyumuqqiwfpñheqrewbrtuseqtuqmupptepeftuwemijlimberhatnatyimpsiwbñheqnopqterpatfiwpptepnetpeerechawten
emu yu muq qiwfp ñheq re wb rtuse qtuq mupp tep ef tuw e

uhp tur qtur pqo sif peq eqej nte ltim uwb hfe qetp ti linh tip ye wb mur ñh tup lu mu hf tin pt eqe seja le tuh nte cene rñh impeq

ufpnuwwumnujorxqohfuxxyewbtetmuhfruqux
ufp nuw wumnu jorx qoh fux xye wb tet mu hf ruqux

hfuwturbmterbjtenemowtemmpemteltereztenexphtewptimjtuwhujltatsepipptewbleippqummuppqaciniwroeepatfsumbjummuppesejalertuhlinhtipmurñhtuphfujumrou
hfuw tu rb mterb jten emow temm pe mte lter ezten exp htew ptim jtuw huj ltat sepi pp te wb le ipp qumm upp qa cin iw roe epatf sumb jumm uppe seja ler tuh linh tip mur ñh tup hf uju mrou

cohfqihohyimpcinihohyimpjorxowuftunpefpterumruyufptuqsejacocitqturuq
cohf qih oh yimp cinih oh yimp jorx ow uf tunp ef pter um ruy ufp tuq seja coc itq tur uq

piwnpturpuwbhfunsiwbewefxewujlunytehtenleqtuhfawmptuhjuwqtunxufpueqtupjihpisemterbtepjopeqenreqytehtenzotereq
piwn pturp uwb hfun si wb ewe fxew uj lun yteh ten leq tuhf awm ptuh juw qtunx ufpue qtup ji hpise mte rb tep jop eqen req yteh ten zot er eq

hftucnastep
hftuc na step

zum

In [29]:
contes

[u'ixqtirraqixdaxdpykhhihwaccixcizixuyrcixzazixraqzixwradiwhictahhizixdteqiayh\xf1razuyrixroyyi\xf1ariwtazixroyyidiqxirtiqiapcizzix\xf1iqqi',
 u'ixzadzazidosodixxicwserixwadziuyrixwsapcodxedihahtiziiyyiqwtiziqtidzod',
 u'tidriz\xf1reqziqiqixjexzidtaqzcizihvaqwctayacwraqiziqtirix\xf1rowzu\xf1aqykwhhoyzihsaxrote\xf1ixhiihapzic',
 u'uyrixraqcidihpeqxtetihcizpeqwyadihedqtirixwhtayyaccihjqadhizihhoyuyrixedytejwtawoxrit',
 u'uyrixtazzihcix\xf1arziossiptaihxeseddosoxzpeqah\xf1ixhizihaxzih',
 u'cixzixjaqwodwtasithihazahptaqihevzadizizixeddarwodhoyahwsqodiedjqtidiwtajtexziqxisecytejixziedpaxzhuyrix',
 u'zivqkdyiziytewvtazixapayyisqtiphiqed\xf1ayhixzied\xf1kyixziytejzixhoyjadihoyqtirix',
 u'zu\xf1aqqodhodxesyessihcodoixqaqwxaqiwadzizixtidroyyi\xf1ixhizihaxzihyacedwtaposjtexziqxipahvtacodeddarwodhoyahvqkdyicodwtatidiqdaxwsicteq',
 u'\xf1reqpeqiqzuedwtawtadqtazodwadziqtirix',
 u'xtiwhizaddosziodixuzeduyrixwadziwsapcodxedihahtiziiyyiqwtiziqtidzodwiyr',
 u'tidrizihwhiz\xf1iq\xf1ixxi\xf1reqsexixwsayy

In [149]:
def getFrontiers(phrase,entropyNgrams,rev=False):
    print phrase
    if rev:
        phrase=phrase[::-1]
        phrase=phrase[1:]+phrase[0]
    result=[]
    for sNgram in entropyNgrams:
        if rev:
            print "G",sNgram
        else:
            print "D",sNgram
        entropy=entropyNgrams[sNgram]
        if rev:
            entropy={k[::-1]:entropy[k] for k in entropy}
        lEntropy=[0]
        for i in range(0,len(phrase)-sNgram-1,1):
            ngram=phrase[i:i+sNgram]
            lEntropy.append(entropy[ngram])
        lEntropy.append(0)
        [frontieres]=[i-1 for i in argrelextrema(np.array(lEntropy), np.greater)]
#        print frontieres
        result=""
        for l in phrase[0:sNgram]:
            result+=l
        for i in range(0,len(phrase)-sNgram-1,1):
            if i in frontieres:
                result+=" "
            result+=phrase[i+sNgram]
        if rev:
            print result[::-1]
        else:
            print result
        print            
    
def getSpaces(texte,entropyRight,entropyLeft):
    for nLigne,ligne in enumerate(texte):
        getFrontiers(ligne,entropyRight)
        getFrontiers(ligne,entropyLeft,rev=True)
        print    

In [151]:
texte[0]

u'ixqtirraqixdaxdpykhhihwaccixcizixuyrcixzazixraqzixwradiwhictahhizixdteqiayh\xf1razuyrixroyyi\xf1ariwtazixroyyidiqxirtiqiapcizzix\xf1iqqi'

In [150]:
getSpaces(texte[0],entropyRight,entropyLeft)

i
D 1
i

D 2
i

D 3
i

D 4
i

D 5
i

D 6
i

D 7
i

D 8
i

D 9
i

D 10
i

D 11
i

D 12
i

D 13
i

D 14
i

D 15
i

D 16
i

D 17
i

D 18
i

D 19
i

D 20
i

D 21
i

D 22
i

D 23
i

D 24
i

D 25
i

D 26
i

D 27
i

D 28
i

D 29
i

D 30
i

D 31
i

D 32
i

D 33
i

D 34
i

i
G 1
i

G 2
i

G 3
i

G 4
i

G 5
i

G 6
i

G 7
i

G 8
i

G 9
i

G 10
i

G 11
i

G 12
i

G 13
i

G 14
i

G 15
i

G 16
i

G 17
i

G 18
i

G 19
i

G 20
i

G 21
i

G 22
i

G 23
i

G 24
i

G 25
i

G 26
i

G 27
i

G 28
i

G 29
i

G 30
i

G 31
i

G 32
i

G 33
i

G 34
i


x
D 1
x

D 2
x

D 3
x

D 4
x

D 5
x

D 6
x

D 7
x

D 8
x

D 9
x

D 10
x

D 11
x

D 12
x

D 13
x

D 14
x

D 15
x

D 16
x

D 17
x

D 18
x

D 19
x

D 20
x

D 21
x

D 22
x

D 23
x

D 24
x

D 25
x

D 26
x

D 27
x

D 28
x

D 29
x

D 30
x

D 31
x

D 32
x

D 33
x

D 34
x

x
G 1
x

G 2
x

G 3
x

G 4
x

G 5
x

G 6
x

G 7
x

G 8
x

G 9
x

G 10
x

G 11
x

G 12
x

G 13
x

G 14
x

G 15
x

G 16
x

G 17
x

G 18
x

G 19
x

G 20
x

G 21
x

G 22
x

G 23
x

G 24
x

G 25
x

G 26
x

G 2

p

G 2
p

G 3
p

G 4
p

G 5
p

G 6
p

G 7
p

G 8
p

G 9
p

G 10
p

G 11
p

G 12
p

G 13
p

G 14
p

G 15
p

G 16
p

G 17
p

G 18
p

G 19
p

G 20
p

G 21
p

G 22
p

G 23
p

G 24
p

G 25
p

G 26
p

G 27
p

G 28
p

G 29
p

G 30
p

G 31
p

G 32
p

G 33
p

G 34
p


y
D 1
y

D 2
y

D 3
y

D 4
y

D 5
y

D 6
y

D 7
y

D 8
y

D 9
y

D 10
y

D 11
y

D 12
y

D 13
y

D 14
y

D 15
y

D 16
y

D 17
y

D 18
y

D 19
y

D 20
y

D 21
y

D 22
y

D 23
y

D 24
y

D 25
y

D 26
y

D 27
y

D 28
y

D 29
y

D 30
y

D 31
y

D 32
y

D 33
y

D 34
y

y
G 1
y

G 2
y

G 3
y

G 4
y

G 5
y

G 6
y

G 7
y

G 8
y

G 9
y

G 10
y

G 11
y

G 12
y

G 13
y

G 14
y

G 15
y

G 16
y

G 17
y

G 18
y

G 19
y

G 20
y

G 21
y

G 22
y

G 23
y

G 24
y

G 25
y

G 26
y

G 27
y

G 28
y

G 29
y

G 30
y

G 31
y

G 32
y

G 33
y

G 34
y


k
D 1
k

D 2
k

D 3
k

D 4
k

D 5
k

D 6
k

D 7
k

D 8
k

D 9
k

D 10
k

D 11
k

D 12
k

D 13
k

D 14
k

D 15
k

D 16
k

D 17
k

D 18
k

D 19
k

D 20
k

D 21
k

D 22
k

D 23
k

D 24
k

D 25
k

D 26
k

D 27
k



D 25
i

D 26
i

D 27
i

D 28
i

D 29
i

D 30
i

D 31
i

D 32
i

D 33
i

D 34
i

i
G 1
i

G 2
i

G 3
i

G 4
i

G 5
i

G 6
i

G 7
i

G 8
i

G 9
i

G 10
i

G 11
i

G 12
i

G 13
i

G 14
i

G 15
i

G 16
i

G 17
i

G 18
i

G 19
i

G 20
i

G 21
i

G 22
i

G 23
i

G 24
i

G 25
i

G 26
i

G 27
i

G 28
i

G 29
i

G 30
i

G 31
i

G 32
i

G 33
i

G 34
i


x
D 1
x

D 2
x

D 3
x

D 4
x

D 5
x

D 6
x

D 7
x

D 8
x

D 9
x

D 10
x

D 11
x

D 12
x

D 13
x

D 14
x

D 15
x

D 16
x

D 17
x

D 18
x

D 19
x

D 20
x

D 21
x

D 22
x

D 23
x

D 24
x

D 25
x

D 26
x

D 27
x

D 28
x

D 29
x

D 30
x

D 31
x

D 32
x

D 33
x

D 34
x

x
G 1
x

G 2
x

G 3
x

G 4
x

G 5
x

G 6
x

G 7
x

G 8
x

G 9
x

G 10
x

G 11
x

G 12
x

G 13
x

G 14
x

G 15
x

G 16
x

G 17
x

G 18
x

G 19
x

G 20
x

G 21
x

G 22
x

G 23
x

G 24
x

G 25
x

G 26
x

G 27
x

G 28
x

G 29
x

G 30
x

G 31
x

G 32
x

G 33
x

G 34
x


u
D 1
u

D 2
u

D 3
u

D 4
u

D 5
u

D 6
u

D 7
u

D 8
u

D 9
u

D 10
u

D 11
u

D 12
u

D 13
u

D 14
u

D 15
u

D 16
u

D 

D 29
z

D 30
z

D 31
z

D 32
z

D 33
z

D 34
z

z
G 1
z

G 2
z

G 3
z

G 4
z

G 5
z

G 6
z

G 7
z

G 8
z

G 9
z

G 10
z

G 11
z

G 12
z

G 13
z

G 14
z

G 15
z

G 16
z

G 17
z

G 18
z

G 19
z

G 20
z

G 21
z

G 22
z

G 23
z

G 24
z

G 25
z

G 26
z

G 27
z

G 28
z

G 29
z

G 30
z

G 31
z

G 32
z

G 33
z

G 34
z


i
D 1
i

D 2
i

D 3
i

D 4
i

D 5
i

D 6
i

D 7
i

D 8
i

D 9
i

D 10
i

D 11
i

D 12
i

D 13
i

D 14
i

D 15
i

D 16
i

D 17
i

D 18
i

D 19
i

D 20
i

D 21
i

D 22
i

D 23
i

D 24
i

D 25
i

D 26
i

D 27
i

D 28
i

D 29
i

D 30
i

D 31
i

D 32
i

D 33
i

D 34
i

i
G 1
i

G 2
i

G 3
i

G 4
i

G 5
i

G 6
i

G 7
i

G 8
i

G 9
i

G 10
i

G 11
i

G 12
i

G 13
i

G 14
i

G 15
i

G 16
i

G 17
i

G 18
i

G 19
i

G 20
i

G 21
i

G 22
i

G 23
i

G 24
i

G 25
i

G 26
i

G 27
i

G 28
i

G 29
i

G 30
i

G 31
i

G 32
i

G 33
i

G 34
i


x
D 1
x

D 2
x

D 3
x

D 4
x

D 5
x

D 6
x

D 7
x

D 8
x

D 9
x

D 10
x

D 11
x

D 12
x

D 13
x

D 14
x

D 15
x

D 16
x

D 17
x

D 18
x

D 19
x

D 20
x

D 

i

G 10
i

G 11
i

G 12
i

G 13
i

G 14
i

G 15
i

G 16
i

G 17
i

G 18
i

G 19
i

G 20
i

G 21
i

G 22
i

G 23
i

G 24
i

G 25
i

G 26
i

G 27
i

G 28
i

G 29
i

G 30
i

G 31
i

G 32
i

G 33
i

G 34
i


z
D 1
z

D 2
z

D 3
z

D 4
z

D 5
z

D 6
z

D 7
z

D 8
z

D 9
z

D 10
z

D 11
z

D 12
z

D 13
z

D 14
z

D 15
z

D 16
z

D 17
z

D 18
z

D 19
z

D 20
z

D 21
z

D 22
z

D 23
z

D 24
z

D 25
z

D 26
z

D 27
z

D 28
z

D 29
z

D 30
z

D 31
z

D 32
z

D 33
z

D 34
z

z
G 1
z

G 2
z

G 3
z

G 4
z

G 5
z

G 6
z

G 7
z

G 8
z

G 9
z

G 10
z

G 11
z

G 12
z

G 13
z

G 14
z

G 15
z

G 16
z

G 17
z

G 18
z

G 19
z

G 20
z

G 21
z

G 22
z

G 23
z

G 24
z

G 25
z

G 26
z

G 27
z

G 28
z

G 29
z

G 30
z

G 31
z

G 32
z

G 33
z

G 34
z


i
D 1
i

D 2
i

D 3
i

D 4
i

D 5
i

D 6
i

D 7
i

D 8
i

D 9
i

D 10
i

D 11
i

D 12
i

D 13
i

D 14
i

D 15
i

D 16
i

D 17
i

D 18
i

D 19
i

D 20
i

D 21
i

D 22
i

D 23
i

D 24
i

D 25
i

D 26
i

D 27
i

D 28
i

D 29
i

D 30
i

D 31
i

D 32
i

D 33
i

D 34
i



u

G 15
u

G 16
u

G 17
u

G 18
u

G 19
u

G 20
u

G 21
u

G 22
u

G 23
u

G 24
u

G 25
u

G 26
u

G 27
u

G 28
u

G 29
u

G 30
u

G 31
u

G 32
u

G 33
u

G 34
u


y
D 1
y

D 2
y

D 3
y

D 4
y

D 5
y

D 6
y

D 7
y

D 8
y

D 9
y

D 10
y

D 11
y

D 12
y

D 13
y

D 14
y

D 15
y

D 16
y

D 17
y

D 18
y

D 19
y

D 20
y

D 21
y

D 22
y

D 23
y

D 24
y

D 25
y

D 26
y

D 27
y

D 28
y

D 29
y

D 30
y

D 31
y

D 32
y

D 33
y

D 34
y

y
G 1
y

G 2
y

G 3
y

G 4
y

G 5
y

G 6
y

G 7
y

G 8
y

G 9
y

G 10
y

G 11
y

G 12
y

G 13
y

G 14
y

G 15
y

G 16
y

G 17
y

G 18
y

G 19
y

G 20
y

G 21
y

G 22
y

G 23
y

G 24
y

G 25
y

G 26
y

G 27
y

G 28
y

G 29
y

G 30
y

G 31
y

G 32
y

G 33
y

G 34
y


r
D 1
r

D 2
r

D 3
r

D 4
r

D 5
r

D 6
r

D 7
r

D 8
r

D 9
r

D 10
r

D 11
r

D 12
r

D 13
r

D 14
r

D 15
r

D 16
r

D 17
r

D 18
r

D 19
r

D 20
r

D 21
r

D 22
r

D 23
r

D 24
r

D 25
r

D 26
r

D 27
r

D 28
r

D 29
r

D 30
r

D 31
r

D 32
r

D 33
r

D 34
r

r
G 1
r

G 2
r

G 3
r

G 4
r

G 5
r

G 6

KeyboardInterrupt: 

In [77]:
debug=False

def getPhrase(ligne,sNgram):
    posEntropie=[]
    for pos in range(len(ligne)-1):
        if pos>=sNgram:
            rightNgram=ligne[pos-sNgram:pos]
            rightEntropie=entropyRight[sNgram][rightNgram]
            if debug: print rightNgram,
        else: 
            rightEntropie=0
            if debug: print "-"*sNgram,
        if pos<=len(ligne)-1-sNgram and pos>0:
            leftNgram=ligne[pos:pos+sNgram]
            leftEntropie=entropyLeft[sNgram][leftNgram]
            if debug: print leftNgram
        else: 
            leftEntropie=0
            if debug: print "-"*sNgram
        posEntropie.append(leftEntropie+rightEntropie)
    [frontieres]=[i for i in argrelextrema(np.array(posEntropie), np.greater)]
    newLigne=""
    for pos,l in enumerate(ligne):
        if pos in frontieres:
            newLigne+=" "
        newLigne+=l
    return newLigne

In [78]:
for ligne in texte[:25]:
    for lNgram in range(4,12):
        sNgram=lNgram-1
        print sNgram
        print getPhrase(ligne,sNgram)

3
lam ais on du chat quipelo te le bal desc eaux la bour sela ven de tta mada me firmi ani une dou blef amille la pa ix du mé na ge la fausse ma it re sse étude de femme al berts ava rus paris veuve and re hous si aux édit eur hé bert et cie succ es se urs 7rue per ro net 71874 hono ré de bal zac bal zac na quità tour sle 16 mai 179 9le jour de la fête desa int hono ré dont on lui donnale nom qui par ut biens on nant et de bon aug ure
4
la ma is on du chat qui pelote leb al desc eaux la bourse la vendet ta ma da me fi rm iani une dou ble fa mil le la pa ix du mé nage la fausse mait res se étude de femme al bert sav ar us par is veuve and re houssi aux édit eur héb ert etc iesu cc ess eurs 7ru ep er ron et 71874 honoré de balzac balz ac naquità tours le 16 mai 1799 lej our de la fête de saint hon oré dont on lui donna le nom qui par ut biens on nant et de bon au gure
5
la mais on du cha tquip elote leb al desc eaux la bour se la vend etta madame fir miani une dou ble fa mille la pa ix